In [8]:
!pip install -U ydata-profiling

Requirement already up-to-date: ydata-profiling in /home/svargasg/de_novicap_test/venv/lib/python3.8/site-packages (4.16.1)


In [ ]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

df = pd.read_csv('de_sample.csv')

In [5]:
profile = ProfileReport(df, title="Profiling Report")

In [7]:
profile.to_file("eda_report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 79.95it/s]
